In [1]:
from toolz import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
%matplotlib widget


import pathlib
from lenses import lens

from collections import Counter

import re
import nltk
import pymorphy2
import fasttext.util

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
nltk.download('popular')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_ru')
nltk.download('tagsets')
nltk.download('stopwords')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to

True

In [3]:
lmap = compose(list, map)
ltake = compose(list, take)

In [4]:
def map_df_ingredients_element(func, df):
    df = df.copy()
    df['ingredients'] = df['ingredients'].map(lambda v: lmap(func, v))
    return df

In [5]:
data_path = pathlib.Path('data')

## Загружаем датафрейм с данными

In [6]:
df = pd.read_json(data_path.joinpath('ready_dataframe.json'))
df = map_df_ingredients_element(compose(str.lower, str.strip), df)
df

,id,title,course,cuisine,ingredients
0,28195,Банановое мороженое с корицей,Выпечка и десерты,Карибская кухня,"[бананы, лимонный сок, молотая корица, ванильн..."
1,28141,Ананасово-кокосовый шербет,Выпечка и десерты,Карибская кухня,"[сахар, кокосовое молоко, корень имбиря, анана..."
2,27929,"Десерт из манго, клубники и текилы",Выпечка и десерты,Карибская кухня,"[клубника, манго, сахар, тертая цедра лайма, т..."
3,28192,Ананас с соусом из манго и рома,Выпечка и десерты,Карибская кухня,"[манго, темный ром, сок лайма, сахар, тертая ц..."
4,28463,Клубничный соус с текилой,Выпечка и десерты,Карибская кухня,"[клубника, сахарная пудра, сок лайма, текила, ..."
...,...,...,...,...,...
41433,43380,Постный борщ с фасолью,Супы,Украинская кухня,"[белая фасоль, овощной бульон, свекла, картофе..."
41434,80446,Суп энгамат,Супы,Шведская кухня,"[цветная капуста, морковь, картофель, лук-поре..."
41435,136820,Гороховый суп с блинчиками,Супы,Шведская кухня,"[горох, репчатый лук, гвоздика, свиная рулька,..."
41436,18014,Суп из брокколи и кростини с сыром бри,Супы,Шведская кухня,"[оливковое масло, репчатый лук, овощной бульон..."


## Работа с ингридиентами

Удобнее работать с листом листов:

In [7]:
ingredients = list(df['ingredients'])
# TODO !!!!!!!!! Временно берем только первые несколько записей 
ingredients = ingredients[:20]
ingredients[:3]

[['бананы', 'лимонный сок', 'молотая корица', 'ванильное мороженое'],
 ['сахар', 'кокосовое молоко', 'корень имбиря', 'ананас', 'сок лайма'],
 ['клубника',
  'манго',
  'сахар',
  'тертая цедра лайма',
  'текила',
  'сок лайма',
  'апельсиновый ликер']]

### Самые часто встречающиеся слова в ингридиентах

In [8]:
ingredients_counter = Counter()
for lst in ingredients:
    ingredients_counter.update(lst)
# first_ingresients, _ = zip(*ingredients_counter.most_common(10))
# first_ingresients = list(first_ingresients)
# first_ingresients
ingredients_counter.most_common(10)

[('сахар', 15),
 ('пшеничная мука', 14),
 ('куриное яйцо', 11),
 ('сливочное масло', 9),
 ('соль', 7),
 ('сахарная пудра', 5),
 ('сок лайма', 4),
 ('сметана', 3),
 ('сода', 3),
 ('корица', 3)]

In [9]:
morph = pymorphy2.MorphAnalyzer()
def lemmatize(word):
    return morph.parse(word)[0].normal_form

it = lmap(compose(tuple, nltk.word_tokenize), ingredients_counter)
it = lens.Each().Each().modify(lemmatize)(it)
it[:10]

[('банан',),
 ('лимонный', 'сок'),
 ('молотый', 'корица'),
 ('ванильный', 'мороженое'),
 ('сахар',),
 ('кокосовый', 'молоко'),
 ('корень', 'имбирь'),
 ('ананас',),
 ('сок', 'лайм'),
 ('клубника',)]

### Токенизация

In [10]:
ingredients_tokenized = lens.Each().Each().modify(nltk.word_tokenize)(ingredients)
ingredients_tokenized[:4]

[[['бананы'],
  ['лимонный', 'сок'],
  ['молотая', 'корица'],
  ['ванильное', 'мороженое']],
 [['сахар'],
  ['кокосовое', 'молоко'],
  ['корень', 'имбиря'],
  ['ананас'],
  ['сок', 'лайма']],
 [['клубника'],
  ['манго'],
  ['сахар'],
  ['тертая', 'цедра', 'лайма'],
  ['текила'],
  ['сок', 'лайма'],
  ['апельсиновый', 'ликер']],
 [['манго'],
  ['темный', 'ром'],
  ['сок', 'лайма'],
  ['сахар'],
  ['тертая', 'цедра', 'лайма'],
  ['ананас']]]

### Стоп-слова и фильтрация

In [11]:
stopwords = nltk.corpus.stopwords.words('russian')

In [12]:
rx = re.compile(r'[a-zA-Zа-яА-Я][a-zA-Zа-яА-Я\d\-]*')

In [13]:
ingredients_filtered = (lens
    .Each().Each().modify(
        lambda tokenized: tuple(filter(
            lambda word: word not in stopwords and rx.match(word),
            tokenized))
    )(ingredients_tokenized)
)
ingredients_filtered[:4]

[[('бананы',),
  ('лимонный', 'сок'),
  ('молотая', 'корица'),
  ('ванильное', 'мороженое')],
 [('сахар',),
  ('кокосовое', 'молоко'),
  ('корень', 'имбиря'),
  ('ананас',),
  ('сок', 'лайма')],
 [('клубника',),
  ('манго',),
  ('сахар',),
  ('тертая', 'цедра', 'лайма'),
  ('текила',),
  ('сок', 'лайма'),
  ('апельсиновый', 'ликер')],
 [('манго',),
  ('темный', 'ром'),
  ('сок', 'лайма'),
  ('сахар',),
  ('тертая', 'цедра', 'лайма'),
  ('ананас',)]]

### Лемматизация

In [14]:
ingredients_lemmatized = lens.Each().Each().Each().modify(lemmatize)(ingredients_filtered)
ingredients_lemmatized[:4]

[[('банан',),
  ('лимонный', 'сок'),
  ('молотый', 'корица'),
  ('ванильный', 'мороженое')],
 [('сахар',),
  ('кокосовый', 'молоко'),
  ('корень', 'имбирь'),
  ('ананас',),
  ('сок', 'лайм')],
 [('клубника',),
  ('манго',),
  ('сахар',),
  ('тёртый', 'цедра', 'лайм'),
  ('текила',),
  ('сок', 'лайм'),
  ('апельсиновый', 'ликёр')],
 [('манго',),
  ('тёмный', 'ром'),
  ('сок', 'лайм'),
  ('сахар',),
  ('тёртый', 'цедра', 'лайм'),
  ('ананас',)]]

In [42]:
all_words_lemmatized = Counter(lens.Each().Each().Each().collect()(ingredients_lemmatized))

all_words_lemmatized_list = list(all_words_lemmatized)

#Добавил UNK и PAD
all_words_lemmatized_list = ['PAD', 'UNK']+all_words_lemmatized_list
all_words_lemmatized_list.sort()
all_words_lemmatized_list[:10]
# dict(all_words.items())

['PAD',
 'UNK',
 'абрикосовый',
 'ананас',
 'апельсиновый',
 'банан',
 'белок',
 'брынза',
 'ванилин',
 'ваниль']

In [16]:
#ОЧЕНЬ ДОЛГО
fasttext.util.download_model('ru', if_exists='ignore')

'cc.ru.300.bin'

In [65]:
ft = fasttext.load_model('cc.ru.300.bin')
dim = ft.get_dimension()
dim

300

### Получаем векторы для слов 

In [66]:
word_embeddings_dict = {}
for word in all_words_lemmatized_list:
    if word  == 'PAD':
        word_embeddings_dict[word] = np.concatenate((np.zeros(dim),[1, 0]))
    elif word  == 'UNK':
        word_embeddings_dict[word] = np.concatenate((np.zeros(dim),[0, 1]))
    else:
        word_embeddings_dict[word] = np.concatenate((ft.get_word_vector(word), [0, 0]))

In [78]:
first(word_embeddings_dict.items())
# np.concatenate((ft.get_word_vector(word), [0, 0])).shape

('PAD',
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    

Расширяем разметрости для PAD и UNK

In [68]:
dim += 2

In [69]:
# word_embeddings_dict

In [70]:
# word_num = [(word, i) for i, word in enumerate(words)]
# word_num[:10]

### Векторы для ингредиентов

In [71]:
all_ingredients_list =  list(set(lens.Each().Each().collect()(ingredients_lemmatized)))
#Добавил UNK и PAD
all_ingredients_list = [('PAD',), ('UNK',)]+all_ingredients_list
all_ingredients_list 
ingredients_numbering_dict = {ingredient: i for i, ingredient in enumerate(all_ingredients_list)}
ltake(6, ingredients_numbering_dict.items())

[(('PAD',), 0),
 (('UNK',), 1),
 (('разрыхлитель',), 2),
 (('клубника',), 3),
 (('сухой', 'дрожжи'), 4),
 (('круглый', 'рис'), 5)]

Для получения вектора для ингридиента просто суммируем векторы для каждого слова в этом ингридиенте:

In [77]:
ingredients_embeddings_dict = {}
for ingr in all_ingredients_list:
    ingredients_embeddings_dict[ingr] = np.zeros(dim)
    for word in ingr:
#         ingredients_embeddings_dict[ingr] += ft.get_word_vector(word) 
        ingredients_embeddings_dict[ingr] += word_embeddings_dict[word] # Должно заработать, когда 
                                                                         # ингредиенты будут лемматизированны
ltake(1, ingredients_embeddings_dict.items())[0]


(('PAD',),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
 

Наконец, для каждого рецепта возьмем его векторизацию как сумму векторов его ингридиентов

In [124]:
ingredients_by_recipe_embedding = [
    sum(ingredients_embeddings_dict[ingredient]
        for ingredient in ingredient_list) 
    for ingredient_list in ingredients_lemmatized
]
len(ingredients_by_recipe_embedding)

20

## Данные для нейросети

Всевозможные метки (course):

In [125]:
all_courses = {name: num for num, name in enumerate(df['course'][~df['course'].duplicated()])}
all_courses

{'Выпечка и десерты': 0,
 'Завтраки': 1,
 'Закуски': 2,
 'Напитки': 3,
 'Основные блюда': 4,
 'Паста и пицца': 5,
 'Салаты': 6,
 'Соусы и маринады': 7,
 'Супы': 8}

In [126]:
df_for_learning = pd.DataFrame(data={
    'id': df['id'][:20],
    'ingredient_embedding': ingredients_by_recipe_embedding,
    'course_mark': [all_courses[course] for course in df['course']][:20]
})
df_for_learning

,id,ingredient_embedding,course_mark
0,28195,"[0.611611146479845, -0.0743011748418212, -0.09...",0
1,28141,"[0.8909059994039126, -0.11706455610692501, -0....",0
2,27929,"[1.08976399153471, -0.4261779775843024, -0.389...",0
3,28192,"[1.1749160811305046, -0.5175759345293045, -0.3...",0
4,28463,"[0.6190221486613154, -0.14342674519866705, -0....",0
5,37902,"[0.5526944873854518, 0.37551341066136956, -0.2...",0
6,18536,"[0.554089829325676, 0.639475641772151, -0.5342...",0
7,35253,"[0.7957627102732658, 0.561975886579603, -0.673...",0
8,47082,"[0.4236516177188605, 0.25815030839294195, -0.1...",0
9,35173,"[0.5946541959419847, 0.9411030139308423, -0.54...",0


In [127]:
# tf.keras.preprocessing.text.one_hot(all_courses, len(all_courses))

## Подготовка BoE

In [128]:
data = np.array(ingredients_by_recipe_embedding)
labels = to_categorical(np.array(df_for_learning['course_mark']), num_classes=len(all_courses))

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

VALIDATION_SPLIT = 0.2
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

## Обучение BoE

In [130]:
sequence_input = layers.Input(shape=(dim,), dtype='int32')
x = layers.Dense(128, activation='relu')(sequence_input)
x = layers.Dense(32, activation='relu')(x)
preds = layers.Dense(len(all_courses), activation='softmax')(x)

model = keras.Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])


In [131]:
model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 302)]             0         
                                                                 
 dense_18 (Dense)            (None, 128)               38784     
                                                                 
 dense_19 (Dense)            (None, 32)                4128      
                                                                 
 dense_20 (Dense)            (None, 9)                 297       
                                                                 
Total params: 43,209
Trainable params: 43,209
Non-trainable params: 0
_________________________________________________________________


In [132]:
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=2, batch_size=128)

Epoch 1/2
1/1 [==============================] - 1s 510ms/step - loss: 2.2982 - acc: 0.0000e+00 - val_loss: 2.0337 - val_acc: 0.7500
Epoch 2/2
1/1 [==============================] - 0s 31ms/step - loss: 1.8905 - acc: 1.0000 - val_loss: 1.8713 - val_acc: 0.7500


## Подготовка к CNN

In [133]:
data = lens.Each().Each().modify(lambda x: ingredients_numbering_dict[x])(ingredients_lemmatized)

In [134]:
data = pad_sequences(data, padding='post', value=0)
labels = to_categorical(np.array(df_for_learning['course_mark']), num_classes=len(all_courses))

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

VALIDATION_SPLIT = 0.2
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [135]:
max_length = len(data[0])
max_length

14

In [136]:
#Эта матрица будет использоваться для слоя Embedding в keras
ingredients_embeddings_matrix = np.zeros((len(all_ingredients_list), dim))
ingredients_embeddings_matrix.shape
for ingr in all_ingredients_list:
    ingredients_embeddings_matrix[ingredients_numbering_dict[ingr]] = ingredients_embeddings_dict[ingr]
ingredients_embeddings_matrix.shape

(74, 302)

In [137]:
embedding_layer = layers.Embedding(len(ingredients_embeddings_dict),
                            dim,
                            weights=[ingredients_embeddings_matrix],
                            input_length=max_length,
                            trainable=False)
# trainable=True позволит до обучаться эмбеддингам на нашем датасете

In [138]:
sequence_input = layers.Input(shape=(max_length,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
preds = layers.Dense(len(all_courses), activation='softmax')(x)

model = keras.Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])



In [139]:
model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 14)]              0         
                                                                 
 embedding_2 (Embedding)     (None, 14, 302)           22348     
                                                                 
 conv1d_20 (Conv1D)          (None, 10, 128)           193408    
                                                                 
 max_pooling1d_15 (MaxPoolin  (None, 2, 128)           0         
 g1D)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 256)               0         
                                                                 
 dense_21 (Dense)            (None, 128)               32896     
                                                           

In [140]:
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=2, batch_size=128)

Epoch 1/2
1/1 [==============================] - 1s 573ms/step - loss: 2.2353 - acc: 0.0000e+00 - val_loss: 1.0907 - val_acc: 1.0000
Epoch 2/2
1/1 [==============================] - 0s 37ms/step - loss: 1.1143 - acc: 1.0000 - val_loss: 0.2664 - val_acc: 1.0000


## Прочее:

In [33]:
it = df.groupby('cuisine').count()
it.sort_values('title', ascending=False).head(30)

,id,title,course,ingredients
cuisine,,,,
Европейская кухня,11699,11699,11699,11699
Русская кухня,7111,7111,7111,7111
Итальянская кухня,4943,4943,4943,4943
Французская кухня,3055,3055,3055,3055
Американская кухня,2192,2192,2192,2192
Авторская кухня,1822,1822,1822,1822
Китайская кухня,840,840,840,840
Греческая кухня,640,640,640,640
Мексиканская кухня,583,583,583,583


In [34]:
a = ft.get_word_vector('кокосовый')-ft.get_word_vector('кокосовое')+ft.get_word_vector('кремовое')-ft.get_word_vector('кремовый')
a@a

0.5043752